In [1]:
#clone repo,make scripts usable for this user,get dependencies
    
import os,sys,pickle
LOADED=os.path.exists('/kaggle/working/csl')
if not LOADED:
    os.system(r'pip install torch torchvision seaborn scikit-learn tqdm')
    os.system(r"git clone https://github.com/1795757/csl/")

#data
import pandas as pd
import numpy as np

#visual
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
tqdm.pandas()

import time
import csl.utils
from csl.utils import toLabel



import csl.unpack_format as unpack_format
import csl.Image_Datasets as Images_Datasets


from torchvision.transforms import ToTensor
from torchvision.io import read_image
import torchvision.models as models
#from torchvision.models import ResNet50_Weights
import torchvision.transforms

import torch
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
import functools
from PIL import Image
RANDOM_STATE=1
import pandas as pd
import pickle

#import torch_xla.core.xla_model as xm
#import torch_xla.distributed.parallel_loader as pl
#import torch_xla.distributed.xla_multiprocessing as xmp



Cloning into 'csl'...


In [2]:
#!rm -r /kaggle/working/csl

In [3]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')#change runtime type

In [4]:
resnet50_pre_transforms=[torchvision.transforms.Resize(size=(224,224))]
resnet50_post_transforms=[torchvision.transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)#bc trained on imagenet
                )]

In [5]:
def get_dataloaders(df,train_size,test_size,dataset_kwargs,batch_size=32,random_state=1):
    cur_categories=df['category_id'].unique()
    convert_category_id={x:i for i,x in enumerate(cur_categories)}
    df['processed']=True
    train_data_df,test_data_df=train_test_split(df,stratify=df["category_id"],train_size=train_size,test_size=test_size,random_state=random_state)
    train_data_df['category_id']=train_data_df['category_id'].apply(toLabel,length=len(cur_categories),convert_category_id=convert_category_id)
    test_data_df['category_id']=test_data_df['category_id'].apply(toLabel,length=len(cur_categories),convert_category_id=convert_category_id)
    train_data=Images_Datasets.Images_Dataset_SAVE(train_data_df,**dataset_kwargs)
    test_data=Images_Datasets.Images_Dataset_SAVE(test_data_df,**dataset_kwargs)
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    return train_dataloader,test_dataloader

def dictify(**kwargs):
    """
    provides an alternative way to write dicts
    """
    return kwargs

In [6]:
def test_hyperparameter_grid_wargs(evaluation_function,evaluation_function_args,evaluation_function_kwargs,hyperparameters,names=None):
    return test_hyperparameter_grid(functools.partial(evaluation_function,*evaluation_function_args,**evaluation_function_kwargs),hyperparameters,names=None)
def test_hyperparameter_grid(evaluation_function,**hyperparameters):
    """
    Evaluate all possible products of hyperparameter ranges
    
    evaluation function(*evaluation_function_args,**evaluation_function_kwargs,**hyperparameter_point
    evaluation_function should return a dict of {name:value} 
    
    hyperparameters in the form {name:iterable}
    evaluation_function should return a dict of {name:value} or names should be provided
    returns a dataframe multiindexed by the hyperparameters with columns for hyperparameters and evaluation_function results 
    evaluation_funciton results are unpacked with each name:value pair corresponding to a column
    
    """
    hyperparameter_names=hyperparameters.keys()
    hyperparameter_ranges=hyperparameters.values()
    
    df=pd.MultiIndex.from_product(iterables=hyperparameter_ranges,names=hyperparameter_names).to_frame()
    df.columns=pd.MultiIndex.from_tuples(tuples=[('hyperparameter',name) for name in hyperparameter_names])
    return test_hyperparameter_df(df,evaluation_function)

def test_hyperparameter_df(df,evaluation_function):
    """
    Applies evaluation function to a dataframe where the columns are hyperparameters and unpacks results
    helper for test_hyperparameter_grid
    """
    results=df.progress_apply(lambda row: evaluation_function(**row['hyperparameter'].to_dict()),axis=1)
    #unpacks results into columns
    names=results.iloc[0].keys()
    for name in names:
       df['results',name]=results.apply(lambda x:x[name])
    return df

In [7]:
def get_resnet50_basic(output_len,linnum=1,dropout=0,lr=1,criterion=nn.CrossEntropyLoss,optimizer=optim.SGD):
  """
  Takes hyperparameters,criterion,optimizer
  and returns model,criterion(),optimizer(model.parameters(),lr=lr)
  
  """
  model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
  linnum=int(linnum)
  model.fc= nn.Sequential(
      nn.Linear(2048,output_len*linnum),#change
      nn.ReLU(inplace=True),
      nn.Dropout(dropout),#change
      nn.Linear(output_len*linnum,output_len)
  )
  criterion= criterion()#loss func
  optimizer = optimizer(model.parameters(), lr=lr)
  return model,criterion,optimizer
def generic_run(train_dataloader,test_dataloader,model,criterion,optimizer,epochs,device):    
  train_loss_track=dict()
  #test_loss_track=dict()
  model.to(device)
  model.train()
  for epoch in range(epochs):
    for i,data in (enumerate(train_dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)#images_df[i]
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss_track[(epoch,i)]=loss.item()
        loss.backward()
        optimizer.step()
  pred,truth=[],[]
  model.eval()
  with torch.no_grad():
    for data in (test_dataloader):
      inputs, labels = data[0].to(device),data[1].to(device)
      outputs = model(inputs)
      #loss = criterion(outputs, labels)
      #test_loss_track[(epoch,i)]=loss.item()
      pred+=tuple(int(t.argmax()) for t in outputs)
      truth+=tuple(int(t.argmax()) for t in labels)
  return {'truth':truth,'pred':pred,'train_loss_track':train_loss_track}#,'test_loss_track':test_loss_track}
from collections.abc import Mapping,Iterable
def eval_factory(run,*run_args,**run_kwargs):
    run=functools.partial(run,*run_args,**run_kwargs)
    def helper(get_model,*get_model_args,**get_model_kwargs):
        model=functools.partial(get_model,*get_model_args,**get_model_kwargs) 
        #model now takes hyperparameters
        #model may return a tuple or dict
        def new(**hyperparameters):
            temp=model(**hyperparameters)
            if isinstance(temp,Mapping):
                return run(**temp)
            elif isinstance(temp,Iterable):
                return run(*temp)
            else:
                raise Exception('the return of model is not an Iterable or Mappable')
        return new
    return helper

In [8]:
ENVIRO="KAGGLE"
def get_data_path(name,abbrev):
    if ENVIRO == "KAGGLE":
        if name == 'enonkishu':
            return '../input/enonkishu-temp/kaggle/working/snapshot-enonkishu-processed/ENO_S1.lila'
        return f"../input/snapshot-safari-preprocessed-v1/snapshot-{name}-processed/{abbrev}_S1.lila"

def get_dataframe(name,abbrev):
    if ENVIRO=='KAGGLE':   
        return pickle.load(open(f'/kaggle/input/snapshot-safari-preprocessed-dataframes/{name}_df.pkl','rb'))
datasets=('karoo','KAR'),('kruger','KRU')#,('kgalagadi','KGA')
results=dict()
#('enonkishu','ENO'),('camdeboo','CDB'),
def accuracy(truth,pred):
    return sklearn.metrics.accuracy_score(truth,pred)

In [9]:
epochs=5
train_ratio=.8
test_ratio=.2
total=1224
for name,abbrev in datasets:
    df=get_dataframe(name,abbrev)
    train_size=train_ratio
    test_size=test_ratio
    if total is not None:
        train_size*=total/len(df)
        test_size*=total/len(df)
    train_dataloader,test_dataloader=get_dataloaders(df,train_size,test_size,          
                                                     dictify(file_extension="pt"
                                                             ,file_path=get_data_path(name,abbrev)
                                                             ,pretensor_transforms_all=resnet50_pre_transforms,
                                                             posttensor_transforms_all=resnet50_post_transforms))
    output_len=len(df['category_id'].unique())
    performance=test_hyperparameter_grid(
        eval_factory(generic_run,train_dataloader,test_dataloader,epochs=epochs,device=device)(get_resnet50_basic,output_len=output_len),
            lr=[0.01*i for i in range(1,5)],
            dropout=[0.025*i for i in range(5)],
            linnum=range(1,5),
            optimizer=[
                optim.NAdam,
                optim.RAdam,
                optim.Adam,
                optim.AdamW,
                optim.Adamax]
            #hyperparameter=iterable
    )
    with open(f'{name}-1224-performance.pkl','wb') as file:
        pickle.dump(performance,file)
    results[name]=performance

  0%|          | 0/400 [00:00<?, ?it/s]

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

  0%|          | 0/400 [00:00<?, ?it/s]